# Baseline experiments

Импорт либ

In [1]:
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split

In [4]:
import gdown
import zipfile
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import cv2 as cv
import seaborn as sns
from itertools import chain
from sklearn.preprocessing import label_binarize
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple
import random
import torch
from torchvision.transforms import ToTensor
from urllib.request import urlopen
import timm
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.svm import SVC
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, roc_curve, auc
from xgboost import XGBClassifier

### EfficientNetV2-S (input 450x450)

In [42]:
imagefolder = "C:/Users/User/images/450/"

Feature Extraction

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    features_only=True,
)

model = model.to(device)
model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    with torch.no_grad():
        outputs = model(images)
        last_feature_map = outputs[-1]  # Get the last feature map
        # Flatten the feature map
        features = last_feature_map.view(last_feature_map.size(0), -1)
        return features

# Создаём DataLoader для последующего Feature extraction'a
dataset = ImageFolder(imagefolder, transform=transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)

num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_size = round(train_test_ratio*len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [05:10<00:00,  3.35it/s]

Train size is 26586
torch.Size([26586, 25600]) torch.Size([6646, 25600])
torch.Size([26586]) torch.Size([6646])


In [6]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_450.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_450.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_450.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_450.csv", y_test_numpy, delimiter=",")

In [2]:
# Импорт фичей


X_numpy = np.genfromtxt("X_train_V2_S_450.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_450.csv", delimiter=",")
y_numpy = np.genfromtxt("y_train_V2_S_450.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_450.csv", delimiter=",")

X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

In [4]:
print(X_train_numpy.shape)
print(X_val_numpy.shape)
print(X_test_numpy.shape)

print(y_train_numpy.shape)
print(y_val_numpy.shape)
print(y_test_numpy.shape)

(21268, 1280)
(5318, 1280)
(6646, 1280)
(21268,)
(5318,)
(6646,)


In [5]:
# Здесь эскпериментируем с модельками

n_neig = [3, 5, 7, 9, 11]
weight = ['uniform', 'distance']
metric = ['euclidean', 'manhattan']


best_params = []
best_accuracy = -1


for n in n_neig:
    for w in weight:
        for m in metric:
            knn = KNN(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_numpy, y_train_numpy)

            y_pred = knn.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            
            print(f"Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = [n, w, m]


knn = KNN(n_neighbors=best_params[0], weights=best_params[1], metric=best_params[2])
knn.fit(X_train_numpy, y_train_numpy)
y_pred = knn.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print("\n The best one's results:")   
print(f"Classes: {knn.classes_}")
print(f"Num of features: {knn.n_features_in_}")
print(f"Accuracy: {accuracy}")
print(f"Best params: n_neighbors={best_params[0]}, weights={best_params[1]}, metric={best_params[2]}")

Accuracy: 0.9428356525009403
Accuracy: 0.9450921399022189
Accuracy: 0.9537420082737872
Accuracy: 0.9558104550582925
Accuracy: 0.9234674689732982
Accuracy: 0.9234674689732982
Accuracy: 0.942647611884167
Accuracy: 0.9441519368183527
Accuracy: 0.8993982700263257
Accuracy: 0.9007145543437383
Accuracy: 0.9285445656261753
Accuracy: 0.928356525009402
Accuracy: 0.8830387363670553
Accuracy: 0.8757051523128996
Accuracy: 0.9135013162843174
Accuracy: 0.9110567882662655
Accuracy: 0.8638585934561865
Accuracy: 0.859157578036856
Accuracy: 0.8997743512598722
Accuracy: 0.8956374576908612

 The best one's results:
Classes: [0. 1. 2. 3. 4. 5. 6.]
Num of features: 1280
Accuracy: 0.9599759253686428
Best params: n_neighbors=3, weights=distance, metric=manhattan


## Обучение на фичах других EfficientNet'ов

### EfficientNet B5

In [12]:
imagefolder = "C:/Users/User/images/456_white/"

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'efficientnet_b5.sw_in12k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 2048])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [07:10<00:00,  2.41it/s]

Train size is 26586
torch.Size([26586, 2048]) torch.Size([6646, 2048])
torch.Size([26586]) torch.Size([6646])


In [14]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V1_B5_456.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V1_B5_456.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V1_B5_456.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V1_B5_456.csv", y_test_numpy, delimiter=",")

In [6]:
# Импорт фичей
X_numpy = np.genfromtxt("X_train_V1_B5_456.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V1_B5_456.csv", delimiter=",")
y_numpy = np.genfromtxt("y_train_V1_B5_456.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V1_B5_456.csv", delimiter=",")

X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

In [7]:
print(X_train_numpy.shape)
print(X_val_numpy.shape)
print(X_test_numpy.shape)

print(y_train_numpy.shape)
print(y_val_numpy.shape)
print(y_test_numpy.shape)

(21268, 2048)
(5318, 2048)
(6646, 2048)
(21268,)
(5318,)
(6646,)


In [8]:
# Здесь эскпериментируем с модельками

n_neig = [3, 5, 7, 9, 11]
weight = ['uniform', 'distance']
metric = ['euclidean', 'manhattan']


best_params = []
best_accuracy = -1


for n in n_neig:
    for w in weight:
        for m in metric:
            knn = KNN(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_numpy, y_train_numpy)

            y_pred = knn.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            
            print(f"Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = [n, w, m]


knn = KNN(n_neighbors=best_params[0], weights=best_params[1], metric=best_params[2])
knn.fit(X_train_numpy, y_train_numpy)
y_pred = knn.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print("\n The best one's results:")   
print(f"Classes: {knn.classes_}")
print(f"Num of features: {knn.n_features_in_}")
print(f"Accuracy: {accuracy}")
print(f"Best params: n_neighbors={best_params[0]}, weights={best_params[1]}, metric={best_params[2]}")

Accuracy: 0.9202707784881534
Accuracy: 0.9219631440391125
Accuracy: 0.9309890936442271
Accuracy: 0.9319292967280932
Accuracy: 0.8941331327566755
Accuracy: 0.8956374576908612
Accuracy: 0.9240315908236179
Accuracy: 0.9210229409552463
Accuracy: 0.8704400150432493
Accuracy: 0.8764573147799924
Accuracy: 0.911432869499812
Accuracy: 0.9138773975178639
Accuracy: 0.8529522376833396
Accuracy: 0.8553967657013914
Accuracy: 0.8990221887927793
Accuracy: 0.905603610379842
Accuracy: 0.8386611508085746
Accuracy: 0.8358405415569763
Accuracy: 0.8918766453553968
Accuracy: 0.8905603610379842

 The best one's results:
Classes: [0. 1. 2. 3. 4. 5. 6.]
Num of features: 2048
Accuracy: 0.931086367739994
Best params: n_neighbors=3, weights=distance, metric=manhattan


### EfficientNet-v2 M

In [17]:
imagefolder = "C:/Users/User/images/480_white/"

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_m.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [06:50<00:00,  2.53it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [19]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_M_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_M_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_M_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_M_480.csv", y_test_numpy, delimiter=",")

In [9]:
# Импорт фичей
X_numpy = np.genfromtxt("X_train_V2_M_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_M_480.csv", delimiter=",")
y_numpy = np.genfromtxt("y_train_V2_M_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_M_480.csv", delimiter=",")

X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

In [10]:
print(X_train_numpy.shape)
print(X_val_numpy.shape)
print(X_test_numpy.shape)

print(y_train_numpy.shape)
print(y_val_numpy.shape)
print(y_test_numpy.shape)

(21268, 1280)
(5318, 1280)
(6646, 1280)
(21268,)
(5318,)
(6646,)


In [11]:
# Здесь эскпериментируем с модельками

n_neig = [3, 5, 7, 9, 11]
weight = ['uniform', 'distance']
metric = ['euclidean', 'manhattan']


best_params = []
best_accuracy = -1


for n in n_neig:
    for w in weight:
        for m in metric:
            knn = KNN(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_numpy, y_train_numpy)

            y_pred = knn.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            
            print(f"Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = [n, w, m]


knn = KNN(n_neighbors=best_params[0], weights=best_params[1], metric=best_params[2])
knn.fit(X_train_numpy, y_train_numpy)
y_pred = knn.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print("\n The best one's results:")   
print(f"Classes: {knn.classes_}")
print(f"Num of features: {knn.n_features_in_}")
print(f"Accuracy: {accuracy}")
print(f"Best params: n_neighbors={best_params[0]}, weights={best_params[1]}, metric={best_params[2]}")

Accuracy: 0.9234674689732982
Accuracy: 0.9313651748777736
Accuracy: 0.9323053779616397
Accuracy: 0.939262880782249
Accuracy: 0.8973298232418202
Accuracy: 0.9078600977811208
Accuracy: 0.9210229409552463
Accuracy: 0.9285445656261753
Accuracy: 0.8740127867619406
Accuracy: 0.8789018427980444
Accuracy: 0.9110567882662655
Accuracy: 0.9133132756675442
Accuracy: 0.8535163595336592
Accuracy: 0.8589695374200828
Accuracy: 0.8939450921399023
Accuracy: 0.9003384731101918
Accuracy: 0.8369687852576156
Accuracy: 0.8459947348627304
Accuracy: 0.8820985332831892
Accuracy: 0.8883038736367055

 The best one's results:
Classes: [0. 1. 2. 3. 4. 5. 6.]
Num of features: 1280
Accuracy: 0.9404152873909118
Best params: n_neighbors=3, weights=distance, metric=manhattan


### EfficientNet-v2 L

In [24]:
imagefolder = "C:/Users/User/images/480_white/"

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_l.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [08:34<00:00,  2.02it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [26]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_L_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_L_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_L_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_L_480.csv", y_test_numpy, delimiter=",")

In [12]:
# Импорт фичей

X_numpy = np.genfromtxt("X_train_V2_L_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_L_480.csv", delimiter=",")
y_numpy = np.genfromtxt("y_train_V2_L_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_L_480.csv", delimiter=",")

X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

In [13]:
print(X_train_numpy.shape)
print(X_val_numpy.shape)
print(X_test_numpy.shape)

print(y_train_numpy.shape)
print(y_val_numpy.shape)
print(y_test_numpy.shape)

(21268, 1280)
(5318, 1280)
(6646, 1280)
(21268,)
(5318,)
(6646,)


In [14]:
# Здесь эскпериментируем с модельками

n_neig = [3, 5, 7, 9, 11]
weight = ['uniform', 'distance']
metric = ['euclidean', 'manhattan']


best_params = []
best_accuracy = -1


for n in n_neig:
    for w in weight:
        for m in metric:
            knn = KNN(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_numpy, y_train_numpy)

            y_pred = knn.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            
            print(f"Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = [n, w, m]


knn = KNN(n_neighbors=best_params[0], weights=best_params[1], metric=best_params[2])
knn.fit(X_train_numpy, y_train_numpy)
y_pred = knn.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print("\n The best one's results:")   
print(f"Classes: {knn.classes_}")
print(f"Num of features: {knn.n_features_in_}")
print(f"Accuracy: {accuracy}")
print(f"Best params: n_neighbors={best_params[0]}, weights={best_params[1]}, metric={best_params[2]}")

Accuracy: 0.9403911244828883
Accuracy: 0.9490409928544565
Accuracy: 0.951109439638962
Accuracy: 0.9567506581421588
Accuracy: 0.9166980067694622
Accuracy: 0.923279428356525
Accuracy: 0.9430236931177134
Accuracy: 0.9479127491538172
Accuracy: 0.897141782625047
Accuracy: 0.9027830011282437
Accuracy: 0.9253478751410305
Accuracy: 0.9345618653629184
Accuracy: 0.8719443399774351
Accuracy: 0.879465964648364
Accuracy: 0.91481760060173
Accuracy: 0.9210229409552463
Accuracy: 0.856713050018804
Accuracy: 0.8619781872884543
Accuracy: 0.8999623918766454
Accuracy: 0.907295975930801

 The best one's results:
Classes: [0. 1. 2. 3. 4. 5. 6.]
Num of features: 1280
Accuracy: 0.9598254589226602
Best params: n_neighbors=3, weights=distance, metric=manhattan


### EfficientNet-V2 S (384x384)

In [37]:
imagefolder = "C:/Users/User/images/384/"

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [04:26<00:00,  3.90it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [39]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_384.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_384.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_384.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_384.csv", y_test_numpy, delimiter=",")

In [15]:
# Импорт фичей
X_numpy = np.genfromtxt("X_train_V2_S_384.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_384.csv", delimiter=",")
y_numpy = np.genfromtxt("y_train_V2_S_384.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_384.csv", delimiter=",")

X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=42)

In [16]:
print(X_train_numpy.shape)
print(X_val_numpy.shape)
print(X_test_numpy.shape)

print(y_train_numpy.shape)
print(y_val_numpy.shape)
print(y_test_numpy.shape)

(21268, 1280)
(5318, 1280)
(6646, 1280)
(21268,)
(5318,)
(6646,)


In [17]:
# Здесь эскпериментируем с модельками

n_neig = [3, 5, 7, 9, 11]
weight = ['uniform', 'distance']
metric = ['euclidean', 'manhattan']


best_params = []
best_accuracy = -1


for n in n_neig:
    for w in weight:
        for m in metric:
            knn = KNN(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_numpy, y_train_numpy)

            y_pred = knn.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            
            print(f"Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = [n, w, m]


knn = KNN(n_neighbors=best_params[0], weights=best_params[1], metric=best_params[2])
knn.fit(X_train_numpy, y_train_numpy)
y_pred = knn.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print("\n The best one's results:")   
print(f"Classes: {knn.classes_}")
print(f"Num of features: {knn.n_features_in_}")
print(f"Accuracy: {accuracy}")
print(f"Best params: n_neighbors={best_params[0]}, weights={best_params[1]}, metric={best_params[2]}")

Accuracy: 0.9368183527641971
Accuracy: 0.9339977435125987
Accuracy: 0.9465964648364047
Accuracy: 0.9458443023693117
Accuracy: 0.9135013162843174
Accuracy: 0.9189544941707409
Accuracy: 0.9292967280932681
Accuracy: 0.9341857841293719
Accuracy: 0.8931929296728093
Accuracy: 0.8960135389244077
Accuracy: 0.9197066566378338
Accuracy: 0.9208349003384732
Accuracy: 0.8770214366303122
Accuracy: 0.8766453553967657
Accuracy: 0.9061677322301617
Accuracy: 0.9076720571643475
Accuracy: 0.859157578036856
Accuracy: 0.8587814968033095
Accuracy: 0.8924407672057164
Accuracy: 0.895449417074088

 The best one's results:
Classes: [0. 1. 2. 3. 4. 5. 6.]
Num of features: 1280
Accuracy: 0.9507974721637075
Best params: n_neighbors=3, weights=distance, metric=euclidean
